# <b>Final Project and Certification: Beginner</b>
Aria Yudhistira Dasopang / Batch 28    



<b>Ringkasan Tugas Project</b>
Dinarasikan OLIST merupakan salah satu marketplace terbesar di Brasil yang menghubungkan berbagai usaha kecil melalui platform penjualannya, dengan pengiriman produk ke pelanggan dilakukan melalui mitra logistik. Sebagai Analis Data, saya ditugaskan menganalisis data dari tim bisnis menggunakan Python untuk analisis dan visualisasi data, berdasarkan metrik-metrik yang telah ditetapkan oleh stakeholders.

## 1. import library dan mengkoneksikan Python dengan SQL

In [2]:
# Import library standar
import sqlite3

# Import library Third-party
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.linear_model import LinearRegression

In [3]:
#membuat koneksi dengan database
conn_olistdb = sqlite3.connect('olistdb.db')

# membuat cursor
cursor_olistdb = conn_olistdb.cursor()

## 2. Merge dataset

Filtering: data transaksi pada dataframe tidak seluruhnya berhasil (delivered) karena beberapa transaksi memiliki status gagal (canceled dan unavailable) dan masih dalam proses (created, approved, invoiced, processing, shipped). Jika transaksi-transaksi ini disertakan dalam analisis yang fokus pada keberhasilan transaksi, hasilnya bisa menjadi kurang akurat. Oleh karena itu dataframe akan difilter berdasarkan status transaksi "delivered".

CREATE TABLE ds_merged AS  
SELECT   
c.customer_id,
c.customer_unique_id,
c.customer_city,
c.customer_state,  
o.order_id,
o.order_status,
o.order_purchase_timestamp,
o.order_approved_at,
o.order_delivered_carrier_date,
o.order_delivered_customer_date,
o.order_estimated_delivery_date,
oi.order_item_id,
oi.product_id,
oi.shipping_limit_date,
oi.price,
oi.freight_value,  
p.product_name_lenght,
p.product_description_lenght,
p.product_photos_qty,
p.product_weight_g,
p.product_length_cm,
p.product_height_cm,
p.product_width_cm,  
pcnt.product_category_name,
pcnt.product_category_name_english,  
pay.payment_sequential,
pay.payment_type,
pay.payment_installments,
pay.payment_value,  
s.seller_id,
s.seller_zip_code_prefix,
s.seller_city,
s.seller_state,  
r.review_id,
r.review_score,
r.review_comment_title,
r.review_comment_message,
r.review_creation_date,
r.review_answer_timestamp  
FROM     olist_customers_dataset c  
LEFT JOIN  
    olist_orders_dataset o ON c.customer_id = o.customer_id  
LEFT JOIN  
    olist_order_items_dataset oi ON o.order_id = oi.order_id  
LEFT JOIN  
    olist_products_dataset p ON oi.product_id = p.product_id  
LEFT JOIN  
    olist_order_reviews_dataset r ON o.order_id = r.order_id  
LEFT JOIN  
    olist_order_payments_dataset pay ON pay.order_id = o.order_id  
LEFT JOIN  
    olist_sellers_dataset s ON oi.seller_id = s.seller_id  
LEFT JOIN  
    product_category_name_translation pcnt ON pcnt.product_category_name = p.product_category_name  
WHERE o.order_status = "delivered";  

In [4]:
# Read dataframe
df_merged = pd.read_sql_query("SELECT * FROM ds_merged", conn_olistdb)

Catatan: proses Cleaning & Transformation Dataframe merupakan hasil eksplorasi bertahap. Rincian Log Cleaning dan Rationale disajikan pada bagian akhir dokumen.

In [5]:
pd.set_option("display.max_columns", 50)

In [6]:
# Konversi tipe data - otomatis
df_merged = df_merged.convert_dtypes()

In [7]:
# Konversi tipe data - manual
int_cols = [
    "order_item_id", "payment_sequential", "payment_installments", 
    "review_score", "product_name_lenght", "product_description_lenght", "product_photos_qty", 
    "product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm", 
]
float_cols = ["price", "freight_value", "payment_value"]
date_cols = [
    "order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date", 
    "order_delivered_customer_date", "order_estimated_delivery_date", "shipping_limit_date", 
    "review_creation_date", "review_answer_timestamp"
]

for col in int_cols:
    df_merged[col] = pd.to_numeric(df_merged[col], errors="coerce").astype("Int64")
for col in float_cols:
    df_merged[col] = pd.to_numeric(df_merged[col], errors="coerce").astype("float")
for col in date_cols:
    df_merged[col] = pd.to_datetime(df_merged[col], errors="coerce")

## 3. Null Identification

Seluruh tabel sebelum digabungkan menjadi tabel "df_merged" tidak memiliki baris, kolom, atau sel NULL atau kosong. Tetapi setelah digabungkan, beberapa sel memiliki nilai NULL dan string kosong. Ini menandakan bahwa kehadiran sel NULL dan string kosong pada df_merged adalah hal yang wajar dan sesuai konteks, dalam artian record tertentu tidak memiliki data untuk kolom-kolom lain yang tidak relevan atau yang struktur tabelnya berbeda.

Meski begitu, sel NULL dan string kosong pada beberapa kolom dan baris perlu ditangani untuk keperluan analisis.
1. Kolom dengan tipe data datetime64[ns] tidak perlu ditangani karena pandas sudah secara otomatis mengubah nilai NULL menjadi "Not a Time" atau NaT. Keberadaan NaT pada kolom-kolom ini tidak mengganggu proses analisis karena pandas akan mengabaikan (tidak meng-error-kan) operasi sel-sel NaT.
2. Kolom dengan tipe data numerik (Int64, Int32, float64) perlu ditangani karena pandas tidak dapat melakukan operasi pada kolom yang memiliki string kosong dan None
3. Beberapa kolom dengan tipe data string perlu distandarisasi dengan string kosong atau NA karena beberapa analisis memerlukan baris-baris yang valid untuk kondisi tertentu. 

In [8]:
# Null handling: Standarisasi null untuk kolom dengan tipe data numerik
numeric_cols = df_merged.select_dtypes(include=['float64', 'Int64']).columns
for col in numeric_cols:
    df_merged[col] = df_merged[col].replace([np.nan, '', None], np.nan)

# Null handling: Standarisasi null untuk kolom dengan tipe data string, khususnya review_id, title dan message
mask = df_merged["review_id"].notna() & (df_merged["review_id"] != "")
df_merged.loc[mask, ["review_comment_title", "review_comment_message"]] = (
    df_merged.loc[mask, ["review_comment_title", "review_comment_message"]].fillna("").apply(lambda x: x.str.strip())
)

In [9]:
# Deduplikasi: beberapa baris data memiliki kombinasi 'order_id', 'order_item_id', dan 'product_id' yang sama, namun berbeda dalam review_id. Ini menandakan bahwa baris data ini memiliki lebih dari 1 review_id, mengindikasikan customer melakukan perubahan atau update review untuk barang dan transaksi yang sama.
df_merged = df_merged.drop_duplicates(subset=['order_id', 'order_item_id', 'product_id'], keep='last')

## 4. Add Columns

In [10]:
# Menambahkan kolom tanggal yang merupakan ekstrak tanggal saja dari kolom "order_purchase_timestamp"
df_merged['tanggal_purchase'] = df_merged['order_purchase_timestamp'].dt.normalize()

# Menambahkan kolom berdasarkan kolom waktu "order_purchase_timestamp"
df_merged["purchase_year"] = df_merged["order_purchase_timestamp"].dt.year
df_merged["purchase_month"] = df_merged["order_purchase_timestamp"].dt.strftime("%B")
df_merged["purchase_day"] = df_merged["order_purchase_timestamp"].dt.strftime("%A")
df_merged["purchase_hour"] = df_merged["order_purchase_timestamp"].dt.hour

# Menambahkan kolom hasil penjumlahan "price" dan "freight_value"
df_merged["total_order_value"] = df_merged["price"] + df_merged["freight_value"]

# Mengubah tipe data "purchase_month" dan "purchase_day" dari object menjadi string
df_merged = df_merged.astype({'purchase_month': 'string', 'purchase_day': 'string'})

In [11]:
# Menyusun ulang urutan kolom untuk mempermudah pemahaman data
indeks_kolom_purchase = df_merged.columns.get_loc("order_purchase_timestamp")
susunan_kolom_baru = (
    df_merged.columns[:indeks_kolom_purchase+1].tolist() +
    ["purchase_year", "purchase_month", "purchase_day", "purchase_hour"] +
    df_merged.columns[indeks_kolom_purchase+1:].tolist()
)
df_merged = df_merged[susunan_kolom_baru]

indeks_kolom_purchase = df_merged.columns.get_loc("freight_value")
susunan_kolom_baru = (
    df_merged.columns[:indeks_kolom_purchase+1].tolist() +
    ["total_order_value"] +
    df_merged.columns[indeks_kolom_purchase+1:].tolist()
)
df_merged = df_merged[susunan_kolom_baru]

In [12]:
df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]

In [13]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [14]:
# Rename kolom
df_merged = df_merged.rename(columns={
    'customer_state': 'address_static',
    'customer_city': 'address_city'
})

In [15]:
Descriptive_statistic = (df_merged[['customer_unique_id', 'address_static', 'address_city', 'order_id', 'product_id', 'product_category_name_english','review_id']]
                         .nunique()
                         .reset_index()
                         .rename(columns={'index': 'Objek', 0: 'Jumlah'}))

In [16]:
Objek_new = {
    'customer_unique_id': 'customer',
    'address_static': 'state',
    'address_city': 'city',
    'order_id': 'order',
    'product_id': 'jenis produk',
    'product_category_name_english': 'kategori produk',
    'review_id': 'review'
}

In [17]:
Descriptive_statistic['Objek'] = Descriptive_statistic['Objek'].replace(Objek_new)

In [18]:
Descriptive_statistic

,Objek,Jumlah
0,customer,93358
1,state,27
2,city,4085
3,order,96478
4,jenis produk,32216
5,kategori produk,71
6,review,95326


In [19]:
jumlah_order_item_id = df_merged['order_item_id'].count()
jumlah_order_item_id

np.int64(110197)

## 5. Customer Demographic

### 5.1 Distribusi customer by state

In [20]:
df_merged_demographic_sorting = df_merged.sort_values(['customer_unique_id', 'order_purchase_timestamp'])

In [21]:
df_merged_demographic_dropsuplicates = df_merged_demographic_sorting.drop_duplicates('customer_unique_id', keep='first')

In [22]:
df_merged_static = df_merged_demographic_dropsuplicates.groupby('address_static')['customer_unique_id'].nunique().reset_index()
df_merged_static = df_merged_static.rename(columns={'customer_unique_id': 'jumlah_customer_static'}).sort_values('jumlah_customer_static', ascending=False)

In [23]:
df_merged_static.head(10)

,address_static,jumlah_customer_static
25,SP,39145
18,RJ,11913
10,MG,10998
22,RS,5167
17,PR,4768
23,SC,3444
4,BA,3158
6,DF,2018
7,ES,1927
8,GO,1894


In [24]:
total_customer = df_merged['customer_unique_id'].nunique()

In [25]:
total_customer

93358

In [26]:
df_merged_static['Persentase'] = (df_merged_static['jumlah_customer_static'] / total_customer * 100).round(2)

In [27]:
df_merged_static.head(5)

,address_static,jumlah_customer_static,Persentase
25,SP,39145,41.93
18,RJ,11913,12.76
10,MG,10998,11.78
22,RS,5167,5.53
17,PR,4768,5.11


In [28]:
# plt.figure(figsize=(12,6))
# ax = sns.barplot(
#     x='address_static',
#     y='jumlah_customer_static',
#     data=df_merged_static.sort_values('jumlah_customer_static', ascending=False))
# plt.title('Jumlah Unique Customer per State')
# ax.bar_label(ax.containers[0], fmt='%.0f', padding=5, rotation=90, fontsize=8, color='blue', style='italic')
# plt.show()

### 5.2 Demografi temporal: pertumbuhan Top 5 State

In [29]:
df_merged["purchase_quarter"] = df_merged["order_purchase_timestamp"].dt.quarter

In [30]:
df_merged["purchase_yearquarter"] = df_merged['purchase_year'].astype(str) + ' Q' + df_merged['purchase_quarter'].astype(str)

In [31]:
df_merged["purchase_yearquarter"] = df_merged["purchase_yearquarter"].astype('string')

In [32]:
df_top5state = (df_merged[df_merged['address_static'].isin(['SP', 'RJ', 'MG', 'RS', 'PR'])]).reset_index()

In [33]:
df_top5state["purchase_yearquarter"] = df_top5state["purchase_yearquarter"].astype('string')

In [34]:
growth_top5state = (df_top5state.groupby(['address_static', 'purchase_yearquarter'])['order_id']
                    .count()
                    .unstack()
                    .fillna(0)
                    .astype(int)
                    .reset_index())

In [35]:
growth_top5state.head(10)

purchase_yearquarter,address_static,2016 Q3,2016 Q4,2017 Q1,2017 Q2,2017 Q3,2017 Q4,2018 Q1,2018 Q2,2018 Q3
0,MG,0,39,806,1149,1570,2481,2825,2583,1463
1,PR,0,30,330,548,679,957,1227,1152,726
2,RJ,0,46,787,1338,1919,2778,2988,2699,1588
3,RS,0,17,331,629,906,1083,1295,1145,728
4,SP,3,107,2112,3944,5474,7983,9816,10311,6698


### 5.3 Distribusi customer by city

In [36]:
df_merged_city = df_merged_demographic_dropsuplicates.groupby('address_city')['customer_unique_id'].nunique().reset_index()
df_merged_city = df_merged_city.rename(columns={'customer_unique_id': 'jumlah_customer_city'}).sort_values('jumlah_customer_city', ascending=False)

In [37]:
df_merged_city['address_city'].nunique()

4085

In [38]:
# # top 10 kota dan persentasenya
# top_10_cities = (
#     df_merged_city
#     .sort_values('jumlah_customer_city', ascending=False)
#     .head(10)
#     .assign(
#         percentage=lambda x: (x['jumlah_customer_city'] / total_customer * 100).round(2)
#     )
#     .reset_index()
# )

# # persentase "Lainnya" (kota di luar top 10)
# other_cities_percentage = 100 - top_10_cities['percentage'].sum()

# # Tambahkan row "Lainnya" ke df
# top_10_with_other = pd.concat([
#     top_10_cities,
#     pd.DataFrame({
#         'address_city': ['Lainnya'],
#         'jumlah_customer_city': [df_merged_city['jumlah_customer_city'].sum() - top_10_cities['jumlah_customer_city'].sum()],
#         'percentage': [other_cities_percentage]
#     })
# ])

# # Hasil akhir
# print(top_10_with_other[['address_city', 'jumlah_customer_city', 'percentage']])

In [39]:
df_merged_city['banyak'] = pd.cut(
    df_merged_city['jumlah_customer_city'],
    bins=[0, 1, 100, float('inf')],
    labels=['1', '2-100', '>100'],
    right=True
)

In [40]:
df_merged_city_classed = df_merged_city['banyak'].value_counts().sort_index().reset_index()
df_merged_city_classed.columns = ['banyak', 'jumlah_customer_cityclass']

In [41]:
df_merged_city_classed

,banyak,jumlah_customer_cityclass
0,1,1173
1,2-100,2782
2,>100,130


In [42]:
(df_merged_city['jumlah_customer_city'] == 1).sum()

np.int64(1173)

In [43]:
# # Buat treemap dengan 2 level: 
# #    - Level 1: Kota individual (top 10)
# #    - Level 2: Agregat "Lainnya"
# fig = px.treemap(
#     top_10_with_other,
#     path=[px.Constant("Total Customer"), 'address_city'],  # Tambahkan root node
#     values='jumlah_customer_city',
#     color='percentage',
#     color_continuous_scale='Blues',
#     title='Distribusi Customer: Top 10 Kota vs Lainnya',
#     hover_data={'percentage': ':.2f%'},
#     branchvalues='total'  # Pastikan nilai "Lainnya" dihitung sebagai total
# )

# # Customisasi tampilan
# fig.update_traces(
#     textinfo='label+value+percent parent',
#     texttemplate='<b>%{label}</b><br>%{value:,} Customer (%{percentParent:.1%})',
#     hovertemplate='<b>%{label}</b><br>Customer: %{value:,}<br>Persentase: %{color:.2f}%'
# )

# fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
# fig.show()

### 5.4 Segmentasi customer repeat order

In [44]:
frekuensi_transaksi_cust = df_merged.groupby('customer_unique_id')['order_id'].nunique().reset_index()
frekuensi_transaksi_cust = frekuensi_transaksi_cust.rename(columns={'order_id': 'banyak_transaksi'}).sort_values('banyak_transaksi', ascending=False)
frekuensi_transaksi_cust.count()

customer_unique_id    93358
banyak_transaksi      93358
dtype: int64

In [45]:
frekuensi_transaksi_cust['klasifikasi_frekuensi'] = frekuensi_transaksi_cust['banyak_transaksi'].apply(
    lambda x: 'Reguler' if x == 1 else ('RO' if 2 <= x <= 4 else 'Loyal')
)

In [46]:
frekuensi_transaksi_cust.head()

,customer_unique_id,banyak_transaksi,klasifikasi_frekuensi
51431,8d50f5eadf50201ccdcedfb9e2ac8455,15,Loyal
22779,3e43e6105506432c953e165fb2acf44c,9,Loyal
36706,6469f99c1f9dfae7733b25662e7f1782,7,Loyal
10060,1b6c7548a2a1f9037c1fd3ddfed95f33,7,Loyal
73921,ca77025e7201e3b30c44b472ff346268,7,Loyal


In [47]:
ringkasan_frekuensi = frekuensi_transaksi_cust.groupby('klasifikasi_frekuensi').agg(
    jumlah_cust_frek = ('customer_unique_id','count'),
    avg_banyak_transaksi = ('banyak_transaksi','mean'),
)

In [48]:
Total_cust = ringkasan_frekuensi['jumlah_cust_frek'].sum()
ringkasan_frekuensi['persentase_frek'] = (ringkasan_frekuensi['jumlah_cust_frek']/ Total_cust * 100).round(2)
ringkasan_frekuensi['persentase_frek'].apply(lambda x: f"{x:.2%}")
print(ringkasan_frekuensi)

                       jumlah_cust_frek  avg_banyak_transaksi  persentase_frek
klasifikasi_frekuensi                                                         
Loyal                                19              6.315789             0.02
RO                                 2782              2.085191             2.98
Reguler                           90557              1.000000            97.00


#### 5.4.2 Customer RO

In [49]:
customer_order_counts = (
    df_merged.groupby(['address_static', 'customer_unique_id'])['order_id']
    .nunique()
    .reset_index(name='jumlah_order')
)

In [50]:
customer_RO = customer_order_counts.query('1 < jumlah_order < 5')

In [51]:
customer_RO

,address_static,customer_unique_id,jumlah_order
15,AC,28989ef45087c96e5a4346e88216c2ba,2
35,AC,5dbba6c01268a8ad43f79157bf4454a0,2
48,AC,8d6512fa0e1c5cdbd789b956de79fe2f,2
74,AC,f7d7fc0a59ef4363fdce6e3aa069d498,2
96,AL,133e20311ec257bda38f1798250d38c5,2
...,...,...,...
93336,TO,cf26c0ed98d29abab5c9fe2643f88d19,2
93341,TO,d2787622df9fd933c164f7cf696a4e2a,2
93348,TO,d75a854dd5534f42bfac3299ac2d0b33,2
93351,TO,dc6502a8fc5ef5239994a36192e7a0ca,2


In [52]:
customer_RO = (customer_RO.groupby('address_static')['customer_unique_id']
               .nunique()
               .to_frame('cust_RO'))

In [53]:
customer_RO = customer_RO.sort_values('cust_RO', ascending=False)

In [54]:
customer_RO = customer_RO.merge(
    df_merged_static,
    on='address_static',
    how= 'left'
)

In [55]:
customer_RO.head()

,address_static,cust_RO,jumlah_customer_static,Persentase
0,SP,1204,39145,41.93
1,RJ,386,11913,12.76
2,MG,315,10998,11.78
3,RS,157,5167,5.53
4,PR,137,4768,5.11


In [56]:
customer_RO['percentage'] = (customer_RO['cust_RO']/customer_RO['jumlah_customer_static'])*100

In [57]:
avg_ro_static = customer_RO['percentage'].mean()
avg_ro_static

np.float64(2.646990387863229)

#### 5.4.3 Customer Loyal

In [58]:
customer_loyal = customer_order_counts[customer_order_counts['jumlah_order'] > 4]

In [59]:
customer_loyal = (customer_loyal.groupby('address_static')['customer_unique_id']
               .nunique()
               .to_frame('cust_loyal'))

In [60]:
ringkasan_frekuensi = frekuensi_transaksi_cust.groupby('klasifikasi_frekuensi').agg(
    jumlah_cust_frek = ('customer_unique_id','count'),
    avg_banyak_transaksi = ('banyak_transaksi','mean'),
)

In [61]:
Total_cust = ringkasan_frekuensi['jumlah_cust_frek'].sum()
ringkasan_frekuensi['persentase_frek'] = (ringkasan_frekuensi['jumlah_cust_frek']/ Total_cust * 100).round(2)
ringkasan_frekuensi['persentase_frek'].apply(lambda x: f"{x:.2%}")
print(ringkasan_frekuensi)

                       jumlah_cust_frek  avg_banyak_transaksi  persentase_frek
klasifikasi_frekuensi                                                         
Loyal                                19              6.315789             0.02
RO                                 2782              2.085191             2.98
Reguler                           90557              1.000000            97.00


### 5.5 Recency

In [62]:
purchase_date_difference = df_merged.groupby('customer_unique_id')['tanggal_purchase'].agg(
    first_purchase = "min",
    last_purchase = "max"
)

In [63]:
purchase_date_difference['days_between'] = (
    purchase_date_difference['last_purchase'] - purchase_date_difference['first_purchase']
).dt.days

In [64]:
customer_order_history = pd.merge(
    purchase_date_difference,
    frekuensi_transaksi_cust,
    on='customer_unique_id'
)
customer_order_history = customer_order_history.sort_values('banyak_transaksi', ascending=False).reset_index(drop=True)

In [65]:
customer_order_history['days_per_transaction'] = (customer_order_history['days_between'] / customer_order_history['banyak_transaksi']).round()

In [66]:
customer_order_history.head(5)

,customer_unique_id,first_purchase,last_purchase,days_between,banyak_transaksi,klasifikasi_frekuensi,days_per_transaction
0,8d50f5eadf50201ccdcedfb9e2ac8455,2017-06-18,2018-08-20,428,15,Loyal,29.0
1,3e43e6105506432c953e165fb2acf44c,2017-09-18,2018-02-27,162,9,Loyal,18.0
2,6469f99c1f9dfae7733b25662e7f1782,2017-09-19,2018-06-28,282,7,Loyal,40.0
3,1b6c7548a2a1f9037c1fd3ddfed95f33,2017-11-13,2018-02-14,93,7,Loyal,13.0
4,ca77025e7201e3b30c44b472ff346268,2017-10-09,2018-06-01,235,7,Loyal,34.0


In [67]:
customer_order_history.query("banyak_transaksi > 1")['days_per_transaction'].mean().round(2)

np.float64(41.52)

In [68]:
loyal_customers = (customer_order_history['banyak_transaksi'] > 1).sum()

In [69]:
loyal_customers

np.int64(2801)

### 5.6 Kesimpulan eksplanatori

a. Penyebaran pelanggan OLIST berdasarkan wilayah
Pelanggan OLIST <b>sangat terkonsentrasi di beberapa State</b>. State São Paulo (SP) mendominasi dengan 41.89% dari total pelanggan, disusul oleh Rio de Janeiro (12.87%) dan Minas Gerais (11.70%). Namun, jika dilihat berdasarkan City, penyebaran pelanggan <b>jauh lebih merata</b>. Hanya 15.57% pelanggan berasal dari kota São Paulo, dan 6.88% dari kota Rio de Janeiro, sedangkan ribuan kota lainnya (lebih dari 4.000) masing-masing memiliki kontribusi pelanggan yang sangat kecil (0–2%).

b. Perilaku repeat order
Sebagian besar pelanggan hanya melakukan sedikit transaksi, yaitu kurang dari 5 kali. Pelanggan yang tergolong "banyak transaksi" (≥5 kali) hanya mencakup 0.02% dari total pelanggan. Jika kriteria repeat order diturunkan menjadi minimal 2 kali, pelanggan yang melakukan repeat order sebanyak lebih dari 1 kali hanya sebanyak 3% dari keseluruhan pelanggan. Ini mengindikasikan <b>tingkat retensi pelanggan yang rendah</b>, dimana OLIST cenderung digunakan sebagai platform transaksi satu kali oleh sebagian besar pengguna.

c. Retensi pelanggan dan pola repeat order
Jika dihitung rata-rata selisih waktu antar transaksi tiap customer, pelanggan cenderung kembali ke platform OLIST setiap 59 hari untuk melakukan transaksi kembali. Hal ini mencerminkan <b>pola kebutuhan pelanggan adalah musiman</b> atau siklus bulanan.

## 6. Order Behavior

### 6.1 Segmentasi customer value

In [70]:
bahan_avg = df_merged.groupby('customer_unique_id').agg(
    jumlah_transaksi = ('order_id', 'nunique'),
    jumlah_item = ('order_item_id', 'count')
)

In [71]:
bahan_avg['avg'] = bahan_avg['jumlah_item']/bahan_avg['jumlah_transaksi']

In [72]:
bahan_avg['kategori'] = bahan_avg['avg'].apply(lambda x: 'Banyak' if x>5 else 'sedikit')

In [73]:
bahan_avg = bahan_avg.sort_values('avg', ascending=False).reset_index()

In [74]:
bahan_avg.head(5)

,customer_unique_id,jumlah_transaksi,jumlah_item,avg,kategori
0,4546caea018ad8c692964e3382debd19,1,21,21.0,Banyak
1,c402f431464c72e27330a67f7b94d4fb,1,20,20.0,Banyak
2,698e1cf81d01a3d389d96145f7fa6df8,1,20,20.0,Banyak
3,11f97da02237a49c8e783dfda6f50e8e,1,15,15.0,Banyak
4,f7ea4eef770a388bd5b225acfc546604,1,14,14.0,Banyak


### 6.2 Waktu pembelian populer

In [75]:
def waktu_jam(J) :
    if 4 < J < 11 :
        return 'Pagi'
    elif 11 <= J < 15 :
        return 'Siang'
    elif 15 <= J < 18 :
        return 'Sore'
    else :
        return 'Malam'

In [76]:
df_waktu_pembelian = df_merged[['order_id', 'order_purchase_timestamp', 'purchase_year', 'purchase_month', 'purchase_day', 'purchase_hour']]
df_waktu_pembelian = df_waktu_pembelian.drop_duplicates(subset='order_id')

In [77]:
df_waktu_pembelian['waktu'] = df_waktu_pembelian['purchase_hour'].apply(waktu_jam)

In [78]:
frekuensi_pembelian_bytime = (
    df_waktu_pembelian.groupby(['purchase_month', 'waktu'])['order_id']
    .nunique()
    .unstack()
).reset_index()
# frekuensi_pembelian_bytime = frekuensi_pembelian_bytime.sort_values('order_id', ascending=False)

In [79]:
frekuensi_pembelian_bytime[['Malam', 'Siang', 'Sore', 'Pagi']].sum()

waktu
Malam    37519
Siang    24880
Sore     18688
Pagi     15391
dtype: int64

In [80]:
frekuensi_pembelian_bytime.to_csv('frekuensi_pembelian_bytime')

In [81]:
df_merged_malam = df_merged[(df_merged['purchase_hour'] >= 18) | (df_merged['purchase_hour'] <= 4)]

In [82]:
kategoripopuler_df_merged_malam = df_merged_malam.groupby('product_category_name_english')['order_id'].nunique().reset_index().sort_values('order_id', ascending=False)

In [83]:
total_transaksi_malam = df_merged_malam['order_id'].nunique()

In [84]:
total_transaksi_malam

37519

In [85]:
kategoripopuler_df_merged_malam['percentage_%'] = ((kategoripopuler_df_merged_malam['order_id'] / total_transaksi_malam) * 100).round(2)

In [86]:
kategori_populer_malam_10 = kategoripopuler_df_merged_malam.head(10)

In [87]:
kategoripopuler_df_merged_malam.head(10)

,product_category_name_english,order_id,percentage_%
7,bed_bath_table,3755,10.01
43,health_beauty,3340,8.90
64,sports_leisure,2866,7.64
39,furniture_decor,2568,6.84
69,watches_gifts,2274,6.06
15,computers_accessories,2218,5.91
49,housewares,2113,5.63
67,telephony,1694,4.52
68,toys,1505,4.01
5,auto,1435,3.82


In [88]:
total_transaksi_top10 = kategori_populer_malam_10['order_id'].sum()
total_transaksi_top10percent = kategori_populer_malam_10['percentage_%'].sum()

In [89]:
kategori_lainnya_malam = pd.DataFrame({
    'product_category_name_english': ['Lainnya'],
    'order_id': [total_transaksi_malam - total_transaksi_top10],
    'percentage_%': [100 - total_transaksi_top10percent]
})

In [90]:
kategori_populer_malam_10lainnya = pd.concat([
    kategori_populer_malam_10,
    kategori_lainnya_malam
], ignore_index=True)

In [91]:
kategori_populer_malam_10lainnya.to_csv('kategori_populer_malam_10lainnya')

### 6.3 Analisis RFM & segmentasi customer

In [92]:
rfm = df_merged.groupby('customer_unique_id').agg({
    'order_id': 'nunique',  # Frequency
    'total_order_value': 'sum'  # Monetary (payment_value biasanya sudah include freight)
}).rename(columns={
    'order_id': 'Frequency',
    'total_order_value': 'Monetary'
}).reset_index()

In [93]:
latest_date = df_merged['order_purchase_timestamp'].max()

In [94]:
rfm_Recency = (latest_date - df_merged.groupby('customer_unique_id')['order_purchase_timestamp'].max()).dt.days

In [95]:
rfm = rfm.merge(rfm_Recency.rename('Recency'), on='customer_unique_id')

In [96]:
rfm.head()

,customer_unique_id,Frequency,Monetary,Recency
0,0000366f3b9a7992bf8c76cfdf3221e2,1,141.90,111
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19,114
2,0000f46a3911fa3c0805444483337064,1,86.22,536
3,0000f6ccb0745a6a4b88665a16c9f078,1,43.62,320
4,0004aac84e0df4da2b147fca70cf8255,1,196.89,287


In [97]:
rfm[rfm['Frequency'] > 1]

,customer_unique_id,Frequency,Monetary,Recency
104,004288347e5e88a27ded2bb23747066c,2,354.37,227
243,00a39521eb40f7012db50455bf083460,2,123.25,87
305,00cc12a6d8b578b8ebd21ea4e2ae8b27,2,126.20,525
404,011575986092c30523ecb71ff10cb473,2,214.90,132
419,011b4adcd54683b480c4d841250a987f,2,236.30,195
...,...,...,...,...
92991,ff03923ad1eb9e32304deb7f9b2a45c9,2,247.06,95
93085,ff44401d0d8f5b9c54a47374eb48c1b8,2,68.00,466
93187,ff8892f7c26aa0446da53d01b18df463,2,330.14,275
93205,ff922bdd6bafcdf99cb90d7f39cea5b3,3,139.60,349


In [98]:
rfm['Segment'] = pd.cut(
    rfm['Monetary'],
    bins = [0,100,200,300, float('inf')],
    labels = ['Low', 'Medium', 'High', 'VIP'],
    right = True
)

In [99]:
segment_counts = rfm['Segment'].value_counts()
segment_counts = segment_counts.reindex(['Low', 'Medium', 'High', 'VIP'])
print(segment_counts)

Segment
Low       43257
Medium    30116
High       9650
VIP       10335
Name: count, dtype: int64


In [100]:
segment_stats = rfm.groupby('Segment', observed=True).agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).round(1)

In [101]:
segment_stats

,Recency,Frequency,Monetary
Segment,,,
Low,239.0,1.0,60.4
Medium,234.9,1.0,142.4
High,233.6,1.1,239.8
VIP,237.4,1.1,600.7


In [102]:
rfm['Segment_Recency'] = np.where(
    rfm['Recency'] > 180, 
    rfm['Segment'].astype(str) + '_Inactive', 
    rfm['Segment'].astype(str) + '_Active'
)

In [103]:
rfm_segment_recency = rfm.groupby('Segment_Recency')['customer_unique_id'].count().reset_index()

In [104]:
rfm_segment_recency

,Segment_Recency,customer_unique_id
0,High_Active,4080
1,High_Inactive,5570
2,Low_Active,17433
3,Low_Inactive,25824
4,Medium_Active,12486
5,Medium_Inactive,17630
6,VIP_Active,4353
7,VIP_Inactive,5982


### 6.4 Payment method & category populer

In [105]:
df_merged['order_id'].nunique()

96478

In [106]:
popular_payment = df_merged.groupby('payment_type')['order_id'].nunique().reset_index()

In [107]:
popular_payment.rename(columns = {'order_id' : 'jumlah'})

,payment_type,jumlah
0,boleto,19191
1,credit_card,72123
2,debit_card,1484
3,voucher,3679


In [108]:
total_order_value_check = df_merged[df_merged['customer_unique_id'] == 'ff922bdd6bafcdf99cb90d7f39cea5b3']

In [109]:
total_order_value_check = total_order_value_check.groupby('customer_unique_id')['total_order_value'].sum()

In [110]:
total_order_value_check

customer_unique_id
ff922bdd6bafcdf99cb90d7f39cea5b3    139.6
Name: total_order_value, dtype: float64

In [111]:
total_customer_value = df_merged.groupby('customer_unique_id')['total_order_value'].sum().reset_index()

In [112]:
total_customer_value['Segment'] = pd.cut(
    total_customer_value['total_order_value'],
    bins = [0,100,200,300, float('inf')],
    labels = ['Low', 'Medium', 'High', 'VIP'],
    right = True
)

In [113]:
monetary_payment = total_customer_value.merge(
    df_merged[['order_id', 'customer_unique_id', 'payment_type']],
    on='customer_unique_id',
    how='left'
).reset_index()

In [114]:
monetary_payment_grouped = monetary_payment.groupby(['payment_type','Segment'], observed=True)['order_id'].count().reset_index()

In [115]:
popular_category = df_merged.groupby('product_category_name_english')['order_id'].count().reset_index()

In [116]:
popular_category.rename(columns = {'order_id' : 'jumlah'}, inplace=True)

In [117]:
popular_category.sort_values('jumlah', ascending=False).head(5)

,product_category_name_english,jumlah
7,bed_bath_table,10953
43,health_beauty,9465
65,sports_leisure,8431
39,furniture_decor,8160
15,computers_accessories,7644


In [118]:
pcne_populer = df_merged.groupby(['purchase_month', 'product_category_name_english'])['order_id'].count().reset_index()

In [119]:
pcne_populer = pcne_populer.sort_values('order_id', ascending = False)

### 6.5 Pertumbuhan tahunan Top 3 category

In [120]:
df_most_populer_pcne = (df_merged[df_merged['product_category_name_english'].isin(['bed_bath_table', 'health_beauty', 'sports_leisure'])]).reset_index()

In [121]:
most_populer_pcne_growth = df_most_populer_pcne.groupby(['product_category_name_english','purchase_year'])['order_id'].count().reset_index()

### 6.6 Category populer berdasarkan customer-value segment

In [122]:
df_most_populer_pcne['Segment'] = pd.cut(
    monetary_payment['total_order_value'],
    bins = [0,100,200,300, float('inf')],
    labels = ['Low', 'Medium', 'High', 'VIP'],
    right = True
)

In [123]:
most_populer_pcne_segment = df_most_populer_pcne.groupby(['Segment','product_category_name_english'], observed = True)['order_id'].count().reset_index()

In [124]:
total_most_populer_pcne_segment = most_populer_pcne_segment['order_id'].sum()

In [125]:
total_most_populer_pcne_segment

np.int64(28849)

In [126]:
df_pcne_segment_lainnya = df_merged[['order_id', 'product_category_name_english']].reset_index()

In [127]:
total_category = df_merged['order_id'].count()

In [128]:
total_category

np.int64(110197)

## 7. Delivery time

### 7.1 Akurasi durasi pengiriman

In [129]:
df_merged['delivery_accuracy'] = df_merged['order_estimated_delivery_date'] - df_merged['order_delivered_customer_date']

In [130]:
akurasi_pengiriman = (df_merged['delivery_accuracy'] >= pd.Timedelta(0)).mean() * 100

In [131]:
akurasi_pengiriman

np.float64(92.08508398595244)

Rata-rata hari pengiriman terlambat

In [132]:
df_pengiriman_telat = df_merged[df_merged['delivery_accuracy'] <= pd.Timedelta(0)].reset_index()

In [133]:
avg_pengiriman_telat = df_merged['delivery_accuracy'].mean()

In [134]:
avg_pengiriman_telat

Timedelta('11 days 07:58:03.714027716')

### 7.2 State dengan pengiriman terlambat terbanyak

In [135]:
static_pengiriman_telat = df_pengiriman_telat.groupby('address_static')['order_id'].nunique().reset_index()

In [136]:
static_pengiriman_telat = static_pengiriman_telat.rename(columns={'order_id' : 'jumlah'}).sort_values('jumlah', ascending= False)

In [137]:
total_pengiriman_telat = static_pengiriman_telat['jumlah'].sum()
static_pengiriman_telat['persentase'] = ((static_pengiriman_telat['jumlah']/total_pengiriman_telat)*100).round()

In [138]:
static_pengiriman_telat.head()

,address_static,jumlah,persentase
25,SP,2387,31.0
18,RJ,1664,21.0
10,MG,637,8.0
4,BA,457,6.0
22,RS,382,5.0


In [139]:
state_lain_yang_telat = df_pengiriman_telat['order_id'].count()

In [140]:
state_lain_yang_telat

np.int64(8714)

### 7.3 Pengiriman terlambat berdasarkan bulanan

In [141]:
df_pengiriman_telat.loc[:, 'bulan'] = df_pengiriman_telat['order_delivered_customer_date'].dt.strftime("%B")
df_pengiriman_telat.loc[:, 'hari'] = df_pengiriman_telat['order_delivered_customer_date'].dt.strftime("%A")

In [142]:
df_pengiriman_telat_ready = df_pengiriman_telat.assign(
    bulan = pd.Categorical(
    df_pengiriman_telat['bulan'],
    categories=month_order,
    ordered=True
    ),
    hari = pd.Categorical(
    df_pengiriman_telat['hari'],
    categories=day_order,
    ordered=True
    )
)

In [143]:
bulan_pengiriman_telat = df_pengiriman_telat_ready.groupby('bulan', observed=False)['order_id'].nunique().reset_index().sort_values('bulan')

In [144]:
bulan_pengiriman_telat

,bulan,order_id
0,January,667
1,February,386
2,March,1144
3,April,1565
4,May,696
5,June,691
6,July,216
7,August,996
8,September,231
9,October,222


In [145]:
# plt.figure(figsize=(12,6))

# sns.lineplot(
#     x = 'bulan',
#     y = 'order_id',
#     data = bulan_pengiriman_telat,
#     color = 'darkblue',
#     label = 'bulan_pengiriman_telat'
# )

# plt.tight_layout()
# plt.legend()
# plt.show()

Pengiriman terlambat bulanan tahunan

In [146]:
df_pengiriman_telat_2017 = df_pengiriman_telat[df_pengiriman_telat['order_estimated_delivery_date'].dt.year == 2017]
df_pengiriman_telat_2018 = df_pengiriman_telat[df_pengiriman_telat['order_estimated_delivery_date'].dt.year == 2018]

In [147]:
df_pengiriman_telat_2017 = df_pengiriman_telat_2017.assign(
    bulan = pd.Categorical(
    df_pengiriman_telat_2017['bulan'],
    categories=month_order,
    ordered=True
    ),
    hari = pd.Categorical(
    df_pengiriman_telat_2017['hari'],
    categories=day_order,
    ordered=True
    )
)

In [148]:
df_pengiriman_telat_2018 = df_pengiriman_telat_2018.assign(
    bulan = pd.Categorical(
    df_pengiriman_telat_2018['bulan'],
    categories=month_order,
    ordered=True
    ),
    hari = pd.Categorical(
    df_pengiriman_telat_2018['hari'],
    categories=day_order,
    ordered=True
    )
)

In [149]:
bulan_pengiriman_telat_2017 = df_pengiriman_telat_2017.groupby('bulan', observed=False)['order_id'].nunique().reset_index().sort_values('bulan')

In [150]:
bulan_pengiriman_telat_2018 = df_pengiriman_telat_2018.groupby('bulan', observed=False)['order_id'].nunique().reset_index().sort_values('bulan')

In [151]:
bulan_pengiriman_telat_2017

,bulan,order_id
0,January,313
1,February,12
2,March,68
3,April,105
4,May,228
5,June,114
6,July,117
7,August,119
8,September,175
9,October,219


In [152]:
bulan_pengiriman_telat_2018

,bulan,order_id
0,January,354
1,February,374
2,March,1076
3,April,1460
4,May,468
5,June,577
6,July,99
7,August,877
8,September,56
9,October,3


### 7.4 Pengiriman terlambat berdasarkan hari

In [153]:
hari_pengiriman_telat = df_pengiriman_telat.groupby('hari', observed=False)['order_id'].nunique().reset_index(name='total').sort_values('hari')
hari_pengiriman_telat_2017 = df_pengiriman_telat_2017.groupby('hari', observed=False)['order_id'].nunique().reset_index(name='2017').sort_values('hari')
hari_pengiriman_telat_2018 = df_pengiriman_telat_2018.groupby('hari', observed=False)['order_id'].nunique().reset_index(name='2018').sort_values('hari')

In [154]:
hari_pengiriman_telat = (hari_pengiriman_telat_2017
                         .merge(hari_pengiriman_telat_2018, on='hari', how='left')
                         .merge(hari_pengiriman_telat, on='hari', how='left')
)

In [155]:
hari_pengiriman_telat

,hari,2017,2018,total
0,Monday,382,944,1327
1,Tuesday,503,985,1489
2,Wednesday,483,1011,1496
3,Thursday,465,974,1439
4,Friday,447,933,1380
5,Saturday,158,398,556
6,Sunday,40,99,139


### 7.5 Rata-rata biaya pengiriman by state

In [156]:
freight_value_static = df_merged.groupby('address_static').agg({
    'freight_value' : 'sum',
    'order_id' : 'count'
}).reset_index()

In [157]:
freight_value_static['average'] = (freight_value_static['freight_value']/freight_value_static['order_id'])

In [158]:
freight_value_static = freight_value_static.sort_values('average', ascending=False)

In [159]:
freight_value_static

,address_static,freight_value,order_id,average
14,PB,25251.73,586,43.091689
21,RR,1982.05,46,43.088043
20,RO,11283.24,273,41.330549
0,AC,3644.36,91,40.047912
16,PI,20457.19,523,39.115086
9,MA,30794.17,800,38.492712
26,TO,11604.86,310,37.435032
24,SE,13714.94,375,36.573173
1,AL,15316.77,427,35.870656
19,RN,18609.12,521,35.718081


### 7.5 Rata-rata weight tiap product dan standard_deviation

In [160]:
df_berat_produk = df_merged[['product_id', 'order_id', 'product_weight_g']]

In [161]:
berat_produk = df_berat_produk.groupby('product_id')['product_weight_g'].agg(
    average_weight = ('mean'),
    std_dev = ('std')
).sort_values('std_dev', ascending=False).reset_index()

In [162]:
berat_produk['average_weight'] = berat_produk['average_weight'].round(0)
berat_produk['std_dev'] = berat_produk['std_dev'].round()

In [163]:
berat_produk[berat_produk['std_dev'] > 0]

,product_id,average_weight,std_dev


## 8. Review Customer

### 8.1 Rata-rata skor review

In [164]:
df_merged['order_id'].nunique()

96478

In [165]:
df_merged['review_id'].nunique()

95326

In [166]:
df_merged['review_id'].isna().sum()

np.int64(827)

In [167]:
test = df_merged[df_merged['review_id'].isna()]
test['order_id'].nunique()

646

In [168]:
test = df_merged[(df_merged['review_id'].notna()) & (df_merged['review_comment_message'] == '')]

In [169]:
cust_rate = df_merged.groupby('order_id')['review_score'].mean().reset_index()

In [170]:
cust_rate = cust_rate.sort_values('review_score', ascending = False)

In [171]:
cust_rate['satisfaction'] = cust_rate['review_score'].apply(
    lambda x: 'satisfied' if pd.notna(x) and x >= 4 
             else ('not satisfied' if pd.notna(x) and x < 4 
                   else 'no review')
)

In [172]:
cust_rate['satisfaction'].value_counts()

satisfaction
satisfied        75648
not satisfied    20184
no review          646
Name: count, dtype: int64

In [173]:
df_merged['review_id'].nunique()

95326

In [174]:
cust_rate

,order_id,review_score,satisfaction
96477,fffe41c64501cc87c801fd61db3f6244,5.0,satisfied
0,00010242fe8c5a6d1ba2dd792cb16214,5.0,satisfied
96476,fffe18544ffabc95dfada21779c9644f,5.0,satisfied
2,000229ec398224ef6ca0657da4fc703e,5.0,satisfied
96475,fffce4705a9662cd70adb13d4a31832d,5.0,satisfied
...,...,...,...
96100,feef9f21cf88daf192de972212d1c5d5,<NA>,no review
96114,fefacc66af859508bf1a7934eab1e97f,<NA>,no review
96157,ff18b518679a92dcb00e16fae93c6d59,<NA>,no review
96254,ff5b7f440481674b38554434638beebf,<NA>,no review


### 8.2 Rasio penyertaan review per transaksi

In [175]:
tabel_order = frekuensi_transaksi_cust[['customer_unique_id', 'banyak_transaksi']]

In [176]:
temp = df_merged.copy()

temp['review_valid'] = (
    temp['review_comment_title'].fillna('').str.strip().ne('') |
    temp['review_comment_message'].fillna('').str.strip().ne('')
)

df_jumlah_review = temp[temp['review_valid']].groupby('customer_unique_id')['order_id'].nunique().reset_index(name='jumlah_review')

df_result = tabel_order.merge(df_jumlah_review, on='customer_unique_id', how='left')
df_result['jumlah_review'] = df_result['jumlah_review'].fillna(0).astype(int)

In [177]:
(((df_result['jumlah_review'].sum()) / (df_result['banyak_transaksi'].sum()))*100).round(2)

np.float64(42.08)

### 8.3 Kelengkapan review by satisfaction

In [178]:
df_satisfaction_review_ratio = df_merged[['order_id', 'customer_unique_id', 'review_id', 'review_score']].reset_index()

In [179]:
df_satisfaction_review_ratio = (df_satisfaction_review_ratio[df_satisfaction_review_ratio['review_id'].notna()]).drop_duplicates(subset='review_id', keep = 'first')

In [180]:
df_satisfaction_review_ratio['satisfaction'] = df_satisfaction_review_ratio['review_score'].apply(
    lambda x: 'satisfied' if pd.notna(x) and x >= 4 
             else ('not satisfied' if pd.notna(x) and x < 4 
                   else 'no review')
)

In [181]:
df_review_kelengkapan = df_merged[['order_id', 'review_id', 'review_comment_title', 'review_comment_message']]

In [182]:
df_review_kelengkapan = (df_review_kelengkapan[df_review_kelengkapan['review_id'].notna()]).drop_duplicates(subset='review_id', keep = 'first')

In [183]:
kondisi_kelengkapan = [
    (np.array(df_review_kelengkapan['review_comment_title'] != '')) & (np.array(df_review_kelengkapan['review_comment_message'] != '')),
    (np.array(df_review_kelengkapan['review_comment_title'] != '')) | (np.array(df_review_kelengkapan['review_comment_message'] != '')),
    (np.array(df_review_kelengkapan['review_comment_title'] == '')) & (np.array(df_review_kelengkapan['review_comment_message'] == ''))
]

jenis_kelengkapan = [
    'Lengkap',
    'Hanya Title/Komen',
    'Hanya Skor'
]

default_value = 'No Review'

In [184]:
df_satisfaction_review_ratio['kelengkapan'] = np.select(
    kondisi_kelengkapan,
    jenis_kelengkapan,
    default=default_value
)

In [185]:
rasio_review = df_satisfaction_review_ratio.groupby(['satisfaction', 'kelengkapan'])['review_id'].count()

In [186]:
rasio_review

satisfaction   kelengkapan      
not satisfied  Hanya Skor            7291
               Hanya Title/Komen    10074
               Lengkap               2666
satisfied      Hanya Skor           47625
               Hanya Title/Komen    20865
               Lengkap               6805
Name: review_id, dtype: int64

## 9. Time Series

### 9.1 Tren penjualan

In [187]:
df_merged['order_purchase_timestamp'] = pd.to_datetime(df_merged['order_purchase_timestamp'])

In [188]:
min_date = df_merged['order_purchase_timestamp'].min()
max_date = df_merged['order_purchase_timestamp'].max()

rentang_transaksi = df_merged[
    (df_merged['order_purchase_timestamp'] == min_date) | 
    (df_merged['order_purchase_timestamp'] == max_date)
]

In [189]:
rentang_transaksi

,customer_id,customer_unique_id,address_city,address_static,order_id,order_status,order_purchase_timestamp,purchase_year,purchase_month,purchase_day,purchase_hour,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,shipping_limit_date,price,freight_value,total_order_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,product_category_name_english,payment_sequential,payment_type,payment_installments,payment_value,seller_id,seller_zip_code_prefix,seller_city,seller_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,tanggal_purchase,purchase_quarter,purchase_yearquarter,delivery_accuracy
35815,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,sao joaquim da barra,SP,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016,September,Thursday,12,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04,1,5a6b04657a4c5ee34285d1e4619a96b4,2016-09-19 23:11:33,44.99,2.83,47.82,34,1036,1,1000,16,16,16,beleza_saude,health_beauty,<NA>,<NA>,<NA>,NaN,ecccfa2bb93b34a3bf033cc5d1dcdc69,81810,curitiba,PR,6916ca4502d6d3bfd39818759d55d536,1,,nao recebi o produto e nem resposta da empresa,2016-10-06,2016-10-07 18:32:28,2016-09-15,3,2016 Q3,-37 days +16:12:22
35816,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,sao joaquim da barra,SP,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016,September,Thursday,12,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04,2,5a6b04657a4c5ee34285d1e4619a96b4,2016-09-19 23:11:33,44.99,2.83,47.82,34,1036,1,1000,16,16,16,beleza_saude,health_beauty,<NA>,<NA>,<NA>,NaN,ecccfa2bb93b34a3bf033cc5d1dcdc69,81810,curitiba,PR,6916ca4502d6d3bfd39818759d55d536,1,,nao recebi o produto e nem resposta da empresa,2016-10-06,2016-10-07 18:32:28,2016-09-15,3,2016 Q3,-37 days +16:12:22
35817,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,sao joaquim da barra,SP,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016,September,Thursday,12,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04,3,5a6b04657a4c5ee34285d1e4619a96b4,2016-09-19 23:11:33,44.99,2.83,47.82,34,1036,1,1000,16,16,16,beleza_saude,health_beauty,<NA>,<NA>,<NA>,NaN,ecccfa2bb93b34a3bf033cc5d1dcdc69,81810,curitiba,PR,6916ca4502d6d3bfd39818759d55d536,1,,nao recebi o produto e nem resposta da empresa,2016-10-06,2016-10-07 18:32:28,2016-09-15,3,2016 Q3,-37 days +16:12:22
102345,898b7fee99c4e42170ab69ba59be0a8b,24ac2b4327e25baf39f2119e4228976a,limeira,SP,35a972d7f8436f405b56e36add1a7140,delivered,2018-08-29 15:00:37,2018,August,Wednesday,15,2018-08-29 15:10:26,2018-08-29 16:57:00,2018-08-30 16:23:36,2018-09-05,1,d04857e7b4b708ee8b8b9921163edba3,2018-08-31 15:10:26,84.99,8.76,93.75,33,1127,2,450,16,4,20,informatica_acessorios,computers_accessories,1,credit_card,1,93.75,9f505651f4a6abe901a56cdc21508025,04102,sao paulo,SP,390c8570648658f80968ae6f77142429,5,,,2018-08-31,2018-09-09 16:49:27,2018-08-29,3,2018 Q3,5 days 07:36:24


In [190]:
tsa = df_merged.groupby('tanggal_purchase')['order_item_id'].count().reset_index()

In [191]:
tsa.count()

tanggal_purchase    612
order_item_id       612
dtype: int64

In [192]:
tsa = tsa.rename(columns={'order_item_id': 'item_terjual'})

In [193]:
tsa['tanggal_purchase'] = pd.to_datetime(tsa['tanggal_purchase'])

In [194]:
tsa.to_csv('tsa')

In [195]:
tsa_weekly = tsa.set_index('tanggal_purchase').resample('W-MON').sum().reset_index()

In [196]:
tsa_weekly['tanggal_purchase'] = pd.to_datetime(tsa_weekly['tanggal_purchase'])

In [197]:
tsa_weekly.tail()

,tanggal_purchase,item_terjual
98,2018-08-06,2370
99,2018-08-13,2107
100,2018-08-20,1997
101,2018-08-27,976
102,2018-09-03,61


In [198]:
tsa_weekly.to_csv('tsa_weekly')

In [229]:
# plt.figure(figsize=(12,6))

# sns.lineplot(
#     x = 'tanggal_purchase',
#     y = 'item_terjual',
#     data = tsa_weekly.sort_values('tanggal_purchase'),
#     color = "red",
#     label = "Tren"
# )

# plt.title('Item Terjual Weekly')
# plt.xticks(rotation=15)
# plt.tight_layout()
# plt.legend()
# plt.show()

In [200]:
# # Pastikan tanggal sudah dalam datetime dan terurut
# tsa_weekly['tanggal_purchase'] = pd.to_datetime(tsa_weekly['tanggal_purchase'])
# tsa_weekly = tsa_weekly.sort_values('tanggal_purchase')

# # Ubah tanggal jadi angka (misal hari ke-n dari tanggal awal)
# tsa_weekly['tanggal_ordinal'] = (tsa_weekly['tanggal_purchase'] - tsa_weekly['tanggal_purchase'].min()).dt.days

# # Model regresi
# X = tsa_weekly[['tanggal_ordinal']]
# y = tsa_weekly['item_terjual']
# model = LinearRegression().fit(X, y)

# # Koefisien tren
# slope = model.coef_[0]
# print(f"Koefisien tren (slope): {slope:.4f}")

# if slope > 0:
#     print("📈 Tren meningkat")
# elif slope < 0:
#     print("📉 Tren menurun")
# else:
#     print("➖ Tren stagnan")


### 9.2 Seasonality

In [201]:
df_2017 = df_waktu_pembelian[df_waktu_pembelian['purchase_year'] == 2017].reset_index()
df_2018 = df_waktu_pembelian[df_waktu_pembelian['purchase_year'] == 2018].reset_index()

In [202]:
df_seasonality_month_2017 = df_2017[['order_id', 'purchase_month']].reset_index()
df_seasonality_month_2018 = df_2018[['order_id', 'purchase_month']].reset_index()
df_seasonality_month_total = df_waktu_pembelian[['order_id', 'purchase_month']].reset_index()

In [203]:
seasonality_month_2017 = df_seasonality_month_2017['purchase_month'].value_counts().reset_index(name='2017')
seasonality_month_2018 = df_seasonality_month_2018['purchase_month'].value_counts().reset_index(name='2018')
seasonality_month_total = df_seasonality_month_total['purchase_month'].value_counts().reset_index(name='total')

In [204]:
seasonality_month = (seasonality_month_2017
                     .merge(seasonality_month_2018, on='purchase_month', how='left')
                     .merge(seasonality_month_total, on='purchase_month', how='left')
                    )

In [205]:
seasonality_month['purchase_month'] = pd.Categorical(
    seasonality_month['purchase_month'],
    categories=month_order,
    ordered=True
)

In [206]:
seasonality_month = seasonality_month.sort_values('purchase_month', ascending=True)

In [207]:
seasonality_month

,purchase_month,2017,2018,total
11,January,750,7069,7819
10,February,1653,6555,8208
8,March,2546,7003,9549
9,April,2303,6798,9101
6,May,3546,6749,10295
7,June,3135,6099,9234
5,July,3872,6159,10031
3,August,4193,6351,10544
4,September,4150,<NA>,4151
2,October,4478,<NA>,4743


In [208]:
# plt.figure(figsize=(12,6))

# sns.lineplot(
#     x = 'purchase_month',
#     y = 'total',
#     data = seasonality_month.sort_values('purchase_month'),
#     color = 'darkblue',
#     label = 'seasonality_month'
# )

# plt.tight_layout()
# plt.legend()
# plt.show()

Seasonality harian

In [216]:
df_seasonality_day_2017 = df_2017[['order_id', 'purchase_day']].reset_index()
df_seasonality_day_2018 = df_2018[['order_id', 'purchase_day']].reset_index()
df_seasonality_day_total = df_waktu_pembelian[['order_id', 'purchase_day']].reset_index()

In [220]:
seasonality_day_2017 = df_seasonality_day_2017['purchase_day'].value_counts().reset_index(name='2017')
seasonality_day_2018 = df_seasonality_day_2018['purchase_day'].value_counts().reset_index(name='2018')
seasonality_day_total = df_seasonality_day_total['purchase_day'].value_counts().reset_index(name='total')

In [221]:
seasonality_day = (seasonality_day_2017
                   .merge(seasonality_day_2018, on='purchase_day', how='left')
                   .merge(seasonality_day_total, on='purchase_day', how='left')
                  )

In [224]:
seasonality_day['purchase_day'] = pd.Categorical(
    seasonality_day['purchase_day'],
    categories=day_order,
    ordered=True
)

In [226]:
seasonality_day = seasonality_day.sort_values('purchase_day')

In [227]:
seasonality_day

,purchase_day,2017,2018,total
1,Monday,6904,8756,15701
0,Tuesday,6920,8529,15503
3,Wednesday,6508,8533,15076
4,Thursday,6286,7995,14323
2,Friday,6658,6988,13685
6,Saturday,4853,5666,10555
5,Sunday,5299,6316,11635


In [228]:
# plt.figure(figsize=(12,6))

# sns.lineplot(
#     x = 'purchase_day',
#     y = 'jumlah_transaksi',
#     data = seasonality_day.sort_values('purchase_day'),
#     color = 'darkblue',
#     label = 'seasonality_day'
# )

# plt.tight_layout()
# plt.legend()
# plt.show()